[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sci-ndp/ndp-ep-py/blob/main/docs/source/tutorials/resource_management.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/sci-ndp/ndp-ep-py/main?filepath=docs%2Fsource%2Ftutorials%2Fresource_management.ipynb)

# Resource Management Tutorial

Welcome to the tutorial on simplified resource management with the NDP EP Python client!

## What You'll Learn

This tutorial covers resource operations that don't require knowing the parent dataset ID:

- **Get Resource**: Retrieve resource details by ID
- **Patch Resource**: Update resource fields by ID
- **Delete Resource**: Remove resources by ID
- **Search Resources**: Find resources across all datasets with filters

## Use Cases

Perfect for:
- **Quick Lookups**: When you have a resource ID but not the dataset ID
- **Bulk Updates**: Finding and updating resources across datasets
- **Resource Discovery**: Searching for resources by format, name, or content
- **Data Cleanup**: Finding and removing outdated resources

## Prerequisites

- Valid NDP EP API credentials (token)
- Basic understanding of datasets and resources in the catalog

In [ ]:
# Install required libraries
!pip install ndp-ep

# Import required modules
import getpass
from ndp_ep import APIClient

print("Libraries installed and imported successfully!")
print("Ready to start Resource Management tutorial")

## 1. Authentication and Client Setup

First, let's configure the client with your credentials.

In [ ]:
# Interactive API configuration
print("Resource Management Configuration")
print("=" * 35)

# Get API base URL
api_url = input("Enter API base URL [http://localhost:8000]: ").strip()
if not api_url:
    api_url = "http://localhost:8000"

print(f"API URL: {api_url}")

# Get API token securely
print("\nAuthentication")
print("Please enter your API token (it will be hidden):")
api_token = getpass.getpass("API Token: ")

if not api_token.strip():
    raise ValueError("API token is required")

print("Credentials configured securely")

In [ ]:
# Initialize the API client
print("Initializing API Client...")

try:
    client = APIClient(base_url=api_url, token=api_token)
    
    # Test connection
    try:
        system_status = client.get_system_status()
        print("API client initialized successfully")
        print(f"Connected to: {api_url}")
    except Exception as e:
        print(f"API connection test failed: {e}")
        print("Continuing - some features may not work")
    
except Exception as e:
    print(f"Failed to initialize client: {e}")
    raise

## 2. Searching Resources

The `search_resources` method allows you to find resources across all datasets using various filters.

In [ ]:
# Search for all CSV resources
print("Searching for CSV Resources")
print("=" * 30)

try:
    results = client.search_resources(format="CSV", limit=10)
    
    print(f"Found {results['count']} CSV resources\n")
    
    for i, resource in enumerate(results['results'][:5], 1):
        print(f"{i}. {resource.get('name', 'Unnamed')}")
        print(f"   ID: {resource.get('id', 'N/A')}")
        print(f"   Dataset: {resource.get('dataset_name', 'N/A')}")
        print(f"   URL: {resource.get('url', 'N/A')[:50]}..." if resource.get('url') else "   URL: N/A")
        print()
        
except Exception as e:
    print(f"Search failed: {e}")

In [ ]:
# Search with general query
print("Searching with General Query")
print("=" * 30)

search_term = input("Enter search term [climate]: ").strip() or "climate"

try:
    results = client.search_resources(q=search_term, limit=10)
    
    print(f"\nFound {results['count']} resources matching '{search_term}'\n")
    
    for i, resource in enumerate(results['results'][:5], 1):
        print(f"{i}. {resource.get('name', 'Unnamed')}")
        print(f"   Format: {resource.get('format', 'N/A')}")
        print(f"   Dataset: {resource.get('dataset_title', resource.get('dataset_name', 'N/A'))}")
        print()
        
except Exception as e:
    print(f"Search failed: {e}")

In [ ]:
# Search with multiple filters
print("Search with Multiple Filters")
print("=" * 30)

try:
    # Find JSON resources with pagination
    results = client.search_resources(
        format="JSON",
        limit=5,
        offset=0
    )
    
    print(f"Found {results['count']} JSON resources")
    print(f"Showing first {len(results['results'])} results\n")
    
    for resource in results['results']:
        print(f"- {resource.get('name', 'Unnamed')} (ID: {resource.get('id', 'N/A')[:8]}...)")
        
except Exception as e:
    print(f"Search failed: {e}")

## 3. Getting Resource Details

Use `get_resource` to retrieve full details of a resource by its ID.

In [ ]:
# Get resource by ID
print("Get Resource by ID")
print("=" * 20)

# Try to get a resource ID from search results or user input
resource_id = None

try:
    # First, search for any resource to get an ID
    search_results = client.search_resources(limit=1)
    if search_results['results']:
        resource_id = search_results['results'][0].get('id')
        print(f"Using resource from search: {resource_id}")
except:
    pass

if not resource_id:
    resource_id = input("Enter resource ID: ").strip()

if resource_id:
    try:
        resource = client.get_resource(resource_id)
        
        print(f"\nResource Details:")
        print(f"  ID: {resource.get('id')}")
        print(f"  Name: {resource.get('name')}")
        print(f"  Format: {resource.get('format')}")
        print(f"  URL: {resource.get('url')}")
        print(f"  Description: {resource.get('description', 'N/A')}")
        print(f"  Package ID: {resource.get('package_id')}")
        
    except ValueError as e:
        print(f"Error: {e}")
else:
    print("No resource ID provided")

## 4. Updating Resources

Use `patch_resource` to update specific fields of a resource without needing the dataset ID.

In [ ]:
# Patch resource example
print("Update Resource by ID")
print("=" * 25)

print("\nExample usage:")
print("""
# Update only the description
result = client.patch_resource(
    resource_id="your-resource-id",
    description="Updated description"
)

# Update multiple fields
result = client.patch_resource(
    resource_id="your-resource-id",
    name="new-name",
    format="JSON",
    description="New description"
)
""")

# Interactive update (optional)
do_update = input("\nWould you like to update a resource? (yes/no): ").strip().lower()

if do_update == 'yes':
    update_resource_id = input("Enter resource ID to update: ").strip()
    new_description = input("Enter new description: ").strip()
    
    if update_resource_id and new_description:
        try:
            result = client.patch_resource(
                update_resource_id,
                description=new_description
            )
            print(f"\nResource updated successfully!")
            print(f"  Name: {result.get('name')}")
            print(f"  Description: {result.get('description')}")
        except ValueError as e:
            print(f"Update failed: {e}")
    else:
        print("Resource ID and description are required")
else:
    print("Skipping resource update")

## 5. Deleting Resources

Use `delete_resource` to remove a resource by its ID.

In [ ]:
# Delete resource example
print("Delete Resource by ID")
print("=" * 25)

print("\nExample usage:")
print("""
# Delete a resource
result = client.delete_resource("your-resource-id")
print(result['message'])
""")

print("\nWARNING: Deletion is permanent!")

# Interactive delete (optional)
do_delete = input("\nWould you like to delete a resource? (yes/no): ").strip().lower()

if do_delete == 'yes':
    delete_resource_id = input("Enter resource ID to delete: ").strip()
    
    if delete_resource_id:
        confirm = input(f"Confirm deletion of {delete_resource_id}? (yes/no): ").strip().lower()
        
        if confirm == 'yes':
            try:
                result = client.delete_resource(delete_resource_id)
                print(f"\n{result.get('message', 'Resource deleted successfully')}")
            except ValueError as e:
                print(f"Deletion failed: {e}")
        else:
            print("Deletion cancelled")
    else:
        print("No resource ID provided")
else:
    print("Skipping resource deletion")

## 6. Comparison with Dataset-Based Methods

The new resource methods simplify operations when you don't have the dataset ID.

In [ ]:
print("Method Comparison")
print("=" * 40)

print("""
NEW SIMPLIFIED METHODS (no dataset_id needed):
----------------------------------------------
client.get_resource(resource_id)
client.patch_resource(resource_id, name=..., description=...)
client.delete_resource(resource_id)
client.search_resources(q=..., format=..., name=...)

EXISTING METHODS (require dataset_id):
--------------------------------------
client.patch_dataset_resource(dataset_id, resource_id, data)
client.delete_dataset_resource(dataset_id, resource_id)

WHEN TO USE EACH:
-----------------
- Use NEW methods when you only have a resource ID
- Use EXISTING methods when working within a specific dataset context
- Use search_resources() to find resources across all datasets
""")

## API Methods Reference

| Method | Endpoint | Description |
|--------|----------|-------------|
| `get_resource(resource_id)` | GET /resource/{id} | Get resource details |
| `patch_resource(resource_id, ...)` | PATCH /resource/{id} | Update resource fields |
| `delete_resource(resource_id)` | DELETE /resource/{id} | Delete a resource |
| `search_resources(...)` | GET /resources/search | Search with filters |

### Search Parameters

| Parameter | Description |
|-----------|-------------|
| `q` | General search query (searches name, url, description) |
| `name` | Filter by resource name (partial match) |
| `url` | Filter by resource URL (partial match) |
| `format` | Filter by format (CSV, JSON, S3, etc.) |
| `description` | Filter by description (partial match) |
| `limit` | Maximum results (default: 100, max: 1000) |
| `offset` | Skip results for pagination |